## 昇思+昇腾开发板：软硬结合玩转大模型实践能力认证（中级）

**环境准备：**

开发者拿到香橙派开发板后，首先需要进行硬件资源确认、镜像烧录以及CANN和MindSpore版本的升级，才可运行该案例，具体如下：

|**香橙派AIpro**|**镜像**|**CANN Toolkit/Kernels**|**MindSpore**|**MindSpore NLP**|
|:-------:|:-------:|:-------:|:-------:|:-------:|
|20T 24G|Ubuntu|8.0.0beta1|2.5.0|0.4分支|

- CANN检查与升级：参考[链接](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/orange_pi/environment_setup.html#3-cann%E5%8D%87%E7%BA%A7)
- MindSpore检查与升级：参考[链接](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/orange_pi/environment_setup.html#4-mindspore%E5%8D%87%E7%BA%A7)
- MindSpore NLP安装命令：
    ```bash
    pip install git+https://github.com/mindspore-lab/mindnlp.git@0.4
    ```

**场景说明：** 在本任务中，我们将实现DeepSeek-R1-Distill-Qwen-1.5B的解码工程，模型在输出logits后，进行包含贪心搜索、top-p采样、temperature采样等选的文本解码策略，最终根据提供的prompt进行文本生成。

**任务目标：** 本任务旨在考核对解码算法的掌握，以及基于MindSpore的开发实践。当前notebook文件已完成了代码工程框架的搭建，并在4个关键模块进行了代码扣空，开发者在昇思官网(https://www.mindspore.cn) 中找到**MindSpore2.5.0版本的接口文档，重点关注mindspore.mint接口文档**，参考文档补全空缺处的代码，保证执行脚本全流程跑通，最终根据开发者自行设计的prompt输出模型生成内容。

**参考文档：**

开发者可在昇思官网(https://www.mindspore.cn) 中找到**MindSpore2.5.0版本的接口文档，重点关注mindspore.mint接口文档**

> 空缺处可通过多行代码实现，不局限于一行

### 模型实例化

模型链接：https://modelers.cn/models/MindSpore-Lab/DeepSeek-R1-Distill-Qwen-1.5B-FP16

In [ ]:
# 导入包
import mindspore
from mindnlp.transformers import AutoTokenizer, AutoModelForCausalLM, StaticCache
from mindspore import mint
import numpy as np

# 开启同步，用于定位问题，调试完毕后建议关闭同步
# mindspore.set_context(pynative_synchronize=True)

# 模型实例化
model_id = "MindSpore-Lab/DeepSeek-R1-Distill-Qwen-1.5B-FP16"
tokenizer = AutoTokenizer.from_pretrained(model_id, mirror="modelers")
model = AutoModelForCausalLM.from_pretrained(model_id, ms_dtype=mindspore.float16, low_cpu_mem_usage=True, mirror="modelers")

### **考点1：Top-p 采样实现**
Top-p 采样是一种根据累积概率动态选择候选词集的解码策略，通过设定阈值p保留概率最高的部分词进行采样，以平衡生成问题的多样性和相关性。

**实现逻辑如下：**<br>
**1.概率排序：** 将模型输出的词汇概率按从高到低排序<br>
**2.累积计算：** 对排序后的概率进行累加，生成累积概率分布<br>
**3.阈值截断：** 保留累积概率首次超过设定值p的最小词集合<br>
**4.子集采样：** 仅从阶段后的词集中依概率随机选取下一个词<br>

**要求：** 请根据上述top-p逻辑，和代码中注释的提示，完成top-p的逻辑实现。

In [ ]:
# Top-p采样实现
def sample_top_p(probs, p=0.9):
    """
    Top-p采样函数，用于生成文本时选择下一个token。
    """
    # 按概率降序排序
    probs_np = probs.asnumpy()
    sorted_indices = np.argsort(-probs_np, axis=-1)
    sorted_probs = np.take_along_axis(probs_np, sorted_indices, axis=-1)
    # 累积计算
    cumulative_probs = np.cumsum(sorted_probs, axis=-1)
     # 构建掩码，阈值截断
    mask = cumulative_probs - sorted_probs > p
    sorted_probs[mask] = 0.0
    sorted_probs /= np.sum(sorted_probs, axis=-1, keepdims=True)
    # 子集采样
    # >>>>>>> 填空：请根据提示，开发top-p采样代码 <<<<<<<
    # 注意香橙派上支持的数据类型为float16和int32
    sorted_probs = ________ # 转换为mindspore.Tensor
    sorted_indices = ________ # 转换为mindspore.Tensor
    new_token_idx = ________ # 获取采样得到的索引，请使用mint.multinomial接口
    # 获取token
    new_token = mindspore.ops.gather(
        sorted_indices, new_token_idx, 
        axis=-1, batch_dims=1
    )
    return new_token

### **考点2： 单token解码函数实现**
在解码过程中，我们首先获取模型单token的前向计算logits，其次根据解码策略（温度采样、top-p采样，或是贪心搜索），获取下一个词。
我们将如上过程写作一个函数decode_one_token。

**要求：** 如下代码已实现了单token的前向计算，请根据如下要求，补齐代码，实现对输出logits的采样和检索。
1. 可自由控制是否执行温度采样，及采样程度，可和top-p采样配合使用
2. 可自由控制是否执行Top-p采样，及采样程度，可和温度采样配合使用
3. 如果温度采样和Top-p采样均不执行，默认执行贪心搜索

**温度采样调节：** 直接使用模型输出的logits进行采样往往会导致概率分布过于尖锐、生成结果缺乏多样性等问题，为此引入温度调节机制，对应公式如下：

$$logits = logits / temperature$$

当温度参数大于1时，会压缩原始logits差异、平缓概率分布、增加生成的多样性，例如：
$$temperature = 1.5$$
$$logits = [5.0, 3.0, 2.0]$$
$$logits_{new} = [3.33, 2.0, 1.33]$$

当温度参数小于1时，会放大原始logits差异、锐化概率分布、增加生成的确定性，例如：
$$temperature = 0.7$$
$$logits = [5.0, 3.0, 2.0]$$
$$logits_{new} = [7.14, 4.29, 2.86]$$

我们鼓励开发者使用不同的temperature进行实验。

In [ ]:
# 单token解码函数实现
def decode_one_tokens_logits(
        model, cur_token, input_pos, 
        cache_position, past_key_values
):
    """单个token的解码函数，返回logits"""
    logits = model(
        cur_token,
        position_ids=input_pos,
        cache_position=cache_position,
        past_key_values=past_key_values,
        return_dict=False,
        use_cache=True
    )[0]
    return logits


def decode_one_tokens(
        model, cur_token, input_pos, 
        cache_position, past_key_values, 
        temperature, top_p
):
    """单个token的解码函数，由logits、温度和Top_p选择合适的token"""
    # 模型前向计算结果
    logits = decode_one_tokens_logits(
        model, cur_token, input_pos,
        cache_position, past_key_values
    )

    # >>>>>>> 填空：补齐单token解码逻辑 <<<<<<<
    # 若使用温度采样 + Top-p采样，请先使用温度采样方法 调整模型输出的logits，
    # 将调整后的logits转化为概率分布后，使用sample_top_p接口得到new_token
    # 若使用贪心搜索，请直接返回模型输出的logits对应概率最高的token即可      
    new_token = ________  # 实现可以为多行代码
    return new_token

### **考点3： prefill阶段实现**
在prefill阶段，模型在首词处理输入序列时，会预先计算并缓存当前所有位置的key、value矩阵，对后续的自回归生成提供加速基础。

prefill阶段逻辑如下：<br>
**1. 静态缓存**：通过创建静态缓存实例化past_key_values<br>
**2. 创建generated_ids：** 创建generated_ids，此为最终生成的内容，后续在decode阶段会不断进行更新<br>
**3. 前向计算：** 初始前向计算获取首个logits<br>
**4. next token生成：** 生成第一个新token<br>
**5. 更新generated_ids：** 将新生成的token更新入generated_ids中<br>

**要求：** 如下代码已完成了静态缓存的实现，请参考上述prefill阶段的逻辑，完成prefill的代码实现。

In [ ]:
# >>>>>>> 填空：请设计一个（或多个）prompt输入 <<<<<<<
prompts = [________]

# tokenization
inputs = tokenizer(prompts, return_tensors="ms", padding=True)

# 生成参数配置
# >>>>>>> 填空：请根据提示，设计合适的参数配置 <<<<<<<
NUM_TOKENS_TO_GENERATE = ________   # 每个输入要生成的token数量
TEMPERATURE = ________  # 温度参数（控制生成多样性）
TOP_P = ________  # Top-p采样阈值


batch_size, seq_length = inputs["input_ids"].shape

# 静态缓存：创建静态缓存（用于加速自回归生成）
past_key_values = StaticCache(
    config=model.config, max_batch_size=len(prompts), max_cache_len=512, dtype=model.dtype
)

# 生成的第一个token的位置
cache_position = mint.arange(seq_length, dtype=mindspore.int32)

# >>>>>>> 填空：补齐prefill逻辑 <<<<<<<
# 创建generated_ids，用于存放最终生成的内容，后续会不断更新
# generated_ids是一个存放token_id的容器，长度为输入序列长度 + 期望生成的token数量 + 1个生成结束的标识符
# generated_ids存放的数据应为：输入序列 + 生成序列 + 结束符，数据类型为int32，每个数据对应词表中的一个token，
# 后续通过解码器将模型生成的token编码序列，解码回自然语言
generated_ids = ________  # 实现可以为多行代码

# 初始前向传播获取首个logits
logits = model(
    **inputs, 
    cache_position=cache_position, 
    past_key_values=past_key_values,
    return_dict=False, 
    use_cache=True
)[0]

# 生成第一个新token，参考前面生成new_token的逻辑
________  # 实现可以为多行代码

# 更新generated_ids
________

### **考点4：decode阶段实现**
在decode阶段，模型在自回归生成每个新的token时，复用之前缓存的KV并仅计算当前步的注意力，从而降低推理复杂度。

decode阶段逻辑如下：<br>
**1. 构建循环：** 构建自回归循环<br>
**2. next token预测：** 生成当前新的token<br>
**3. 更新cache position**<br>

Qwen模型推理机制属于step-by-step，每次仅会生成一个新token，且每个token的生成都需要前文所有的token信息，表示公式如下：
$$token_{n+1} = decode\_one\_tokens(token_{1...n})$$

**要求：** 请参考上述逻辑补齐代码，完成decode阶段的代码实现。

In [ ]:
# 自回归生成循环
cache_position = mindspore.tensor([seq_length + 1])
# >>>>>>> 填空：补齐自回归循环 <<<<<<<
# 请调用decode_one_tokens接口获取下一个token，并在generated_ids容器中完成对应位置token_id的更新
for _ in range(1, NUM_TOKENS_TO_GENERATE):
    ________  # 实现可以为多行代码

# 将最终生成的tokens（generated_ids）转换为文本
text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(text)